In [4]:
import os
import random
from pathlib import Path
from tifffile import imread, imwrite
import numpy as np
from tqdm import tqdm


In [15]:
# Constants
ROOT_DIR = Path("/media/localadmin/EGlab20TB-5/TMA_final_run_01/input/")
TILE_SIZE_UM = 200
RESOLUTION = 0.17  # µm/pixel
TILE_SIZE_PX = int(TILE_SIZE_UM / RESOLUTION)  # ~1176 pixels

In [16]:
BATCH = "2_annotators_CD2"
OUT_MARKER_DIR = Path("../data/" + BATCH + "/tiles_marker")
OUT_DAPI_DIR = Path("../data/" + BATCH + "/tiles_DAPI")
OUT_MARKER_DIR.mkdir(parents=True, exist_ok=True)
OUT_DAPI_DIR.mkdir(parents=True, exist_ok=True)

In [17]:
# Get all subfolders (sample names)
sample_folders = sorted([f for f in ROOT_DIR.iterdir() if f.is_dir()])
random.seed(42)
random.shuffle(sample_folders)
selected_folders = sample_folders[:100]

In [18]:
from scipy.ndimage import binary_dilation, label
import numpy as np
import cv2
from scipy.signal import find_peaks
from tifffile import imread, imwrite

def remove_bright_aggregates(img, threshold_factor=3.0, visualize=False):
    """
    Detects and suppresses bright aggregates in a grayscale image.

    Args:
        img (np.ndarray): 2D grayscale image (float32 or uint16).
        threshold_factor (float): How many times brighter a peak must be than the 99th percentile to be considered artifact.
        visualize (bool): If True, show intermediate plots (requires matplotlib).

    Returns:
        cleaned_img (np.ndarray): Image with bright spots suppressed.
        mask (np.ndarray): Boolean mask of removed regions.
    """
    # Normalize image to [0, 1] for histogram stability
    img_norm = img.astype(np.float32)
    img_norm /= img_norm.max()

    # Histogram
    hist, bin_edges = np.histogram(img_norm.flatten(), bins=512, range=(0, 1))
    
    # Peak detection in histogram
    peaks, _ = find_peaks(hist, distance=20)

    if len(peaks) == 0:
        return img, np.zeros_like(img, dtype=bool)

    # Estimate "normal" high signal level
    signal_level = np.percentile(img_norm, 99)

    # Check the rightmost peak
    brightest_bin_idx = peaks[np.argmax(bin_edges[peaks])]
    brightest_bin_val = bin_edges[brightest_bin_idx]

    if brightest_bin_val > threshold_factor * signal_level:
        print(f"Artifact detected: Brightest bin value {brightest_bin_val:.4f} exceeds threshold {threshold_factor * signal_level:.4f}")
        # Artifact suspected: mask extreme high intensity
        thresh = brightest_bin_val - 0.01
        mask = img_norm > thresh
        # Find objects in the mask
        labeled_mask, num_features = label(mask)

        # Extend each object by 100px
        extended_mask = binary_dilation(labeled_mask, structure=np.ones((101, 101)))

        # Re-find objects
        relabeled_mask, num_features = label(extended_mask)
        # Directly use the relabeled mask as the final mask
        mask = relabeled_mask > 0
        
        cleaned = img.copy()
        median_without_spot = np.median(cleaned[~mask])
        
        # Replace bright aggregates with median value
        # Create a fake background with random noise around the median value
        noise = np.random.normal(loc=median_without_spot, scale=0.0001, size=mask.sum())
        cleaned[mask] = noise
        
    else:
        # No artifact detected
        mask = np.zeros_like(img, dtype=bool)
        cleaned = img
        
    return cleaned

In [ ]:
print(f"Selected {len(selected_folders)} samples.")
marker = "CD4"

for folder in tqdm(selected_folders):
    sample_name = folder.name
    marker_path = folder / marker + ".tiff"
    dapi_path = folder / "DAPI.tiff"
    
    if not marker_path.exists() or not dapi_path.exists():
        print(f"Missing " + marker + " or DAPI in {sample_name}. Skipping.")
        continue

    try:
        img_marker = imread(marker_path)
        img_dapi = imread(dapi_path)
    except Exception as e:
        print(f"Error reading {sample_name}: {e}")
        continue

    H, W = img_marker.shape
    if img_marker.shape != img_dapi.shape:
        print(f"Shape mismatch in {sample_name}. Skipping.")
        continue

    max_x = W - TILE_SIZE_PX
    max_y = H - TILE_SIZE_PX
    if max_x <= 0 or max_y <= 0:
        print(f"Image too small in {sample_name}. Skipping.")
        continue

    # Select a random (x, y) that fits the tile
    x = random.randint(0, max_x)
    y = random.randint(0, max_y)

    # Print the sample name and position
    print(f"Processing {sample_name} at position ({x}, {y})...")

    tile_marker = img_marker[y:y + TILE_SIZE_PX, x:x + TILE_SIZE_PX]
    tile_dapi = img_dapi[y:y + TILE_SIZE_PX, x:x + TILE_SIZE_PX]
    
    # Remove the 99.9th percentile
    percentile_99 = np.percentile(tile_dapi, 99.99)
    tile_dapi[tile_dapi > percentile_99] = percentile_99

    # Scale the image between 0 and 1
    tile_dapi = tile_dapi / percentile_99

    # Remove the 99.9th percentile
    percentile_99 = np.percentile(tile_marker, 99.9)
    tile_marker[tile_marker > percentile_99] = percentile_99
    
    # Scale the image between 0 and 1
    tile_marker = tile_marker / percentile_99

    tile_marker_cleaned = remove_bright_aggregates(tile_marker, threshold_factor=2.5, visualize=True)

    base_name = f"{sample_name}_x{x}_y{y}.tiff"
    marker_tile_path = OUT_MARKER_DIR / base_name.replace(".tiff", "_" + marker + ".tiff")
    dapi_tile_path = OUT_DAPI_DIR / base_name.replace(".tiff", "_DAPI.tiff")

    imwrite(marker_tile_path, tile_marker_cleaned)
    imwrite(dapi_tile_path, tile_dapi)

print("✅ Done saving 100 random tiles.")

Selected 50 samples.


  0%|          | 0/50 [00:00<?, ?it/s]

Processing MF155_early_IB_TMA1 at position (1555, 1215)...
Processing MF066_early_TMA2 at position (445, 479)...
Artifact detected: Brightest bin value 0.9648 exceeds threshold 0.1318


  4%|▍         | 2/50 [00:06<02:34,  3.23s/it]

Processing MF156_late_1_TMA1 at position (4744, 6027)...
Processing MF010_early_1_TMA1 at position (1110, 499)...
Artifact detected: Brightest bin value 0.9688 exceeds threshold 0.0562


...

Processing MF205_late_TMA1 at position (4543, 2452)...
Processing MF036_late_2_TMA2 at position (848, 3845)...
Processing MF157_early_TMA2 at position (1100, 1083)...


 92%|█████████▏| 46/50 [00:53<00:02,  1.93it/s]

Processing MF208_late_TMA2 at position (945, 876)...
Processing MF003_late_1_TMA1 at position (4532, 636)...
Processing AD079_TMA1 at position (2231, 1154)...
Processing MF025_late_2_TMA1 at position (1238, 1725)...
Artifact detected: Brightest bin value 0.9902 exceeds threshold 0.0209


100%|██████████| 50/50 [00:59<00:00,  1.20s/it]

Processing MF072_early_TMA1 at position (5878, 1404)...
Processing AD143_TMA1 at position (1667, 2815)...
Processing MF023_late_2_TMA2 at position (2162, 2070)...
✅ Done saving 50 random tiles.


In [3]:
import os
from pathlib import Path
import numpy as np
from skimage.io import imread, imsave
from tqdm import tqdm

# Set root input/output
input_root = Path("/media/localadmin/EG_Lab_9TB/TMA_final_run_01/input")
output_root = Path("/media/localadmin/EG_Lab_9TB/TMA_final_run_01/input_processed")

# Find all .tiff files under each patient_id subfolder
tiff_files = list(input_root.rglob("*.tiff"))

print(f"Found {len(tiff_files)} TIFF files to process.")



Found 2740 TIFF files to process.


In [4]:

for src_path in tqdm(tiff_files):
    rel_path = src_path.relative_to(input_root)
    dst_path = output_root / rel_path
    dst_path.parent.mkdir(parents=True, exist_ok=True)


    # Print processing patient ID and file name
    print(f"Processing {src_path.name} for patient ID {rel_path.parts[0]}...")

    print(f"Processing {src_path.name}...")
    img = imread(str(src_path)).astype(np.float16)  # Assuming 16-bit input

    if src_path.name == "DAPI.tiff":
        # Remove the 99.99th percentile
        percentile_99_99 = np.percentile(img, 99.99)
        img[img > percentile_99_99] = percentile_99_99
        
        # Scale the image between 0 and 1
        img = img / percentile_99_99
        cleaned = np.clip(img, 0, 1)  # Ensure values are between 0 and 1
    else:
        # Remove the 99.9th percentile
        percentile_99 = np.percentile(img, 99.9)
        img[img > percentile_99] = percentile_99
        
        # Scale the image between 0 and 1
        img = img / percentile_99
        
        cleaned = remove_bright_aggregates(img, threshold_factor=2.5, visualize=False)
        cleaned = np.clip(cleaned, 0, 1)  # Ensure values are between 0 and 1

    imsave(str(dst_path), (cleaned).astype(np.float16))  # Save as 16-bit again


  0%|          | 0/2740 [00:00<?, ?it/s]

Processing DAPI.tiff for patient ID AD145_TMA1...
Processing DAPI.tiff...


  0%|          | 1/2740 [00:00<43:40,  1.05it/s]

Processing S100A4.tiff for patient ID AD145_TMA1...
Processing S100A4.tiff...


  0%|          | 2/2740 [00:02<55:54,  1.23s/it]

...

100%|█████████▉| 2738/2740 [28:11:53<00:52, 26.26s/it]

Processing EZH2.tiff for patient ID MF015_early_TMA1...
Processing EZH2.tiff...


100%|█████████▉| 2739/2740 [28:11:54<00:18, 18.77s/it]

Processing CD5.tiff for patient ID MF015_early_TMA1...
Processing CD5.tiff...


100%|██████████| 2740/2740 [28:11:56<00:00, 37.05s/it]
